<a href="https://colab.research.google.com/github/Samoed/CourseWork2/blob/main/TestApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install -U bert-extractive-summarizer
!pip3 install -U transformers

Requirement already up-to-date: bert-extractive-summarizer in ./.local/lib/python3.8/site-packages (0.7.1)
Requirement already up-to-date: transformers in ./.local/lib/python3.8/site-packages (4.4.2)


In [ ]:
!unzip embeddingsShort.zip

In [2]:
import requests
import torch
from transformers import BertTokenizer, BertModel, BertConfig
from tqdm import tqdm
from summarizer import Summarizer

from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtWidgets import QMessageBox
import sys

import pickle
import re
import math
import os

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#print(torch.cuda.get_device_name(0))

custom_config = BertConfig.from_pretrained('DeepPavlov/rubert-base-cased')
custom_config.output_hidden_states=True
custom_tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
custom_model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased', config=custom_config)
custom_model.to('cuda')
custom_model.eval()

modelSum = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)

In [3]:
path = 'data/embeddingsPickle'
directories = os.listdir(path)
emb = {}
for file in directories:
    with open(path+'/'+file, "rb") as f:
        arr = pickle.load(f)
        emb[file.split('.')[0]] = arr

In [6]:
with open("data/TestDoc", 'r') as f:
    text = f.read()
    text = "".join([i.replace('\n',' ').replace('\t',' ') for i in text])
text
%time embeddings = modelSum.run_embeddings(text, num_sentences=10)

In [16]:
error = {}
for name, arr in tqdm(emb.items()):
    sum = 0
    avg = 0
    mae = 0
    rmae = 0
    mse = 0
    rmse = 0
    for i in range(len(embeddings)):
        for arrRow in range(len(arr)):
            for j in range(768):
                sum += embeddings[i][j] - arr[arrRow][j]
                mae += abs(embeddings[i][j] - arr[arrRow][j])
                rmae += abs(embeddings[i][j] - arr[arrRow][j])**0.5
                mse += (embeddings[i][j] - arr[arrRow][j])**2
                rmse += ((embeddings[i][j] - arr[arrRow][j])**2)**0.5
    error[sum] = name

100%|██████████| 60/60 [01:10<00:00,  1.18s/it]


In [17]:
error[min(error.keys())]

' Кычкин Алексей Владимирович'

In [2]:
class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(600, 600)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        
        self.textBox = QtWidgets.QTextEdit(self.centralwidget)
        self.textBox.setGeometry(QtCore.QRect(10, 50, 571, 411))
        self.textBox.setObjectName("textBox")
        
        self.readyButton = QtWidgets.QPushButton(self.centralwidget)
        self.readyButton.setGeometry(QtCore.QRect(590, 440, 89, 25))
        self.readyButton.setObjectName("readyButton")
        self.readyButton.clicked.connect(self.clickMethod)
        
        self.comboBox = QtWidgets.QComboBox(self.centralwidget)
        self.comboBox.setGeometry(QtCore.QRect(590, 50, 171, 25))
        self.comboBox.setObjectName("comboBox")
        
        self.label = QtWidgets.QLabel(self.centralwidget)
        self.label.setGeometry(QtCore.QRect(10, 30, 141, 17))
        self.label.setObjectName("label")
        
        self.label_2 = QtWidgets.QLabel(self.centralwidget)
        self.label_2.setGeometry(QtCore.QRect(560, 30, 231, 17))
        self.label_2.setObjectName("label_2")
        
        MainWindow.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(MainWindow)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 800, 22))
        self.menubar.setObjectName("menubar")
        MainWindow.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        self.statusbar.setObjectName("statusbar")
        MainWindow.setStatusBar(self.statusbar)

        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "MainWindow"))
        self.readyButton.setText(_translate("MainWindow", "Готово"))
        self.label.setText(_translate("MainWindow", "Введите текст:"))
        self.label_2.setText(_translate("MainWindow", "Выберите функцию расстояния"))
    
    def ComputeError(embText, embProf):
        error = {}
        for name, arr in tqdm(embProf.items()):
            sum = 0
            avg = 0
            mae = 0
            rmae = 0
            mse = 0
            rmse = 0
            for i in range(len(embText)):
                for arrRow in range(len(arr)):
                    for j in range(768):
                        sum += embeddings[i][j] - arr[arrRow][j]
                        mae += abs(embeddings[i][j] - arr[arrRow][j])
                        rmae += abs(embeddings[i][j] - arr[arrRow][j])**0.5
                        mse += (embeddings[i][j] - arr[arrRow][j])**2
                        rmse += ((embeddings[i][j] - arr[arrRow][j])**2)**0.5
            error[sum] = name
        return error
    
    def clickMethod(self):
        text = self.textBox.toPlainText()
        text = "".join([i.replace('\n',' ').replace('\t',' ') for i in text])
        embText = modelSum.run_embeddings(text, num_sentences=10)
        error = ComputeError(embText, emb)
        QMessageBox.about(MainWindow, "Title", error[min(error.keys())])

        
def loadEmb():
    path = 'data/embeddingsPickle'
    directories = os.listdir(path)
    emb = {}
    for file in directories:
        with open(path+'/'+file, "rb") as f:
            arr = pickle.load(f)
            emb[file.split('.')[0]] = arr
    return emb

def loadModel():
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    #print(torch.cuda.get_device_name(0))

    custom_config = BertConfig.from_pretrained('DeepPavlov/rubert-base-cased')
    custom_config.output_hidden_states=True
    custom_tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
    custom_model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased', config=custom_config)
    custom_model.to('cuda')
    custom_model.eval()
    return Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)
            
if __name__ == "__main__":
    
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.show()
    
    emb = loadEmb()
    model = loadModel()
    
    sys.exit(app.exec_())

SystemExit: 0

/home/samoed/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
